In [1]:
import for_report as fr

In [2]:
# выгрузка отчета из базы данных PostgreSQL в DataFrame

sql = '''WITH ads_t AS (
SELECT 
    created_at AS Date, 
    d_utm_source AS UTM_source, 
    d_utm_medium AS UTM_medium, 
    d_utm_campaign AS UTM_campaign, 
    SUM(m_clicks) AS Clicks_number, 
    SUM(m_cost) AS adv_expenses 
FROM ads 
WHERE m_clicks!=0
GROUP BY Date, UTM_source, UTM_medium, UTM_campaign),

leads_t AS (
SELECT 
    created_at AS Date, 
    d_utm_source AS UTM_source, 
    d_utm_medium AS UTM_medium, 
    d_utm_campaign AS UTM_campaign, 
    COUNT(lead_id) AS Leads_number 
FROM ads ad JOIN (
        SELECT lead_created_at, 
            MIN(lead_id) AS lead_id, 
            d_lead_utm_source, 
            d_lead_utm_medium, 
            d_lead_utm_campaign, 
            d_lead_utm_content, 
            d_lead_utm_term, 
            client_id
        FROM leads
        WHERE client_id IS NOT NULL
        GROUP BY lead_created_at, d_lead_utm_source, d_lead_utm_medium, 
            d_lead_utm_campaign, d_lead_utm_content, d_lead_utm_term, client_id
        UNION
        SELECT lead_created_at, 
            lead_id, 
            d_lead_utm_source, 
            d_lead_utm_medium, 
            d_lead_utm_campaign, 
            d_lead_utm_content, 
            d_lead_utm_term, 
            client_id
        FROM leads
        WHERE client_id IS NULL) l 
    ON ad.created_at=l.lead_created_at 
    AND ad.d_utm_source=l.d_lead_utm_source 
    AND ad.d_utm_medium=l.d_lead_utm_medium 
    AND ad.d_utm_campaign=l.d_lead_utm_campaign 
    AND ad.d_utm_content=l.d_lead_utm_content 
    AND COALESCE(ad.d_utm_term, '0') = COALESCE(l.d_lead_utm_term, '0') 
GROUP BY Date, UTM_source, UTM_medium, UTM_campaign),

purch_data AS ( 
SELECT 
    DISTINCT FIRST_VALUE(lead_created_at) OVER w AS Date, 
    FIRST_VALUE(l.d_lead_utm_source) OVER w AS UTM_source, 
    FIRST_VALUE(l.d_lead_utm_medium) OVER w AS UTM_medium, 
    FIRST_VALUE(l.d_lead_utm_campaign) OVER w AS UTM_campaign, 
    FIRST_VALUE(lead_id) OVER w AS lead_id, 
    purchase_id, 
    m_purchase_amount 
FROM ads ad JOIN leads l 
    ON ad.created_at=l.lead_created_at 
    AND ad.d_utm_source=l.d_lead_utm_source 
    AND ad.d_utm_medium=l.d_lead_utm_medium 
    AND ad.d_utm_campaign=l.d_lead_utm_campaign 
    AND ad.d_utm_content=l.d_lead_utm_content 
    AND COALESCE(ad.d_utm_term, '0') = COALESCE(l.d_lead_utm_term, '0') 
    JOIN purchases pu 
    USING(client_id) 
WHERE pu.m_purchase_amount!=0 AND 
    pu.purchases_created_at BETWEEN l.lead_created_at AND l.lead_created_at+15 
WINDOW w AS (PARTITION BY purchase_id ORDER BY lead_created_at DESC)),

purch_t AS (
SELECT 
    Date, 
    UTM_source, 
    UTM_medium, 
    UTM_campaign, 
    COUNT(purchase_id) AS Sales_number, 
    SUM(m_purchase_amount) AS Revenue 
FROM purch_data 
GROUP BY Date, UTM_source, UTM_medium, UTM_campaign)

SELECT 
    Date, 
    UTM_source, 
    UTM_medium, 
    UTM_campaign,
    Clicks_number, 
    adv_expenses, 
    COALESCE(Leads_number, 0) AS Leads_number, 
    COALESCE(Sales_number, 0) AS Sales_number, 
    COALESCE(Revenue, 0) AS Revenue,
    COALESCE(adv_expenses / Leads_number, 0)::NUMERIC(10,2) AS CPL,
    COALESCE(Revenue / adv_expenses, 0)::NUMERIC(10,2) AS ROAS
FROM ads_t LEFT JOIN leads_t USING(Date, UTM_source, UTM_medium, UTM_campaign)
    LEFT JOIN purch_t USING(Date, UTM_source, UTM_medium, UTM_campaign)
    ORDER BY Date;'''

p = fr.Postgres()
df_report = p.get_df(sql)
df_report

,date,utm_source,utm_medium,utm_campaign,clicks_number,adv_expenses,leads_number,sales_number,revenue,cpl,roas
0,2022-01-03,yandex,cpc,48306435,1,11.952,0,0,0,0.00,0.00
1,2022-01-03,yandex,cpc,48306450,10,190.128,1,0,0,190.13,0.00
2,2022-01-04,yandex,cpc,48306435,1,11.364,0,0,0,0.00,0.00
3,2022-01-04,yandex,cpc,48306450,29,617.616,1,2,9990,617.62,16.18
4,2022-01-05,yandex,cpc,48306450,32,726.192,1,1,290,726.19,0.40
...,...,...,...,...,...,...,...,...,...,...,...
1105,2022-09-13,yandex,cpc,48306473,2,23.328,1,1,12495,23.33,535.62
1106,2022-09-14,yandex,cpc,48306435,14,449.028,2,1,9999,224.51,22.27
1107,2022-09-14,yandex,cpc,48306450,48,1856.208,4,1,10400,464.05,5.60
1108,2022-09-14,yandex,cpc,48306473,1,33.444,0,0,0,0.00,0.00


In [3]:
# загрузка отчета в базу данных PostgreSQL

table_name = 'new_report'
sql_report = f'''CREATE TABLE IF NOT EXISTS {table_name} (
    Date date, 
    utm_source text, 
    utm_medium text, 
    utm_campaign text, 
    clicks_number int, 
    adv_expenses double precision, 
    leads_number int, 
    sales_number int, 
    revenue int, 
    CPL numeric(10,2), 
    ROAS numeric(10,2))'''

p = fr.Postgres()
p.export_df(sql_report, df_report, table_name)

Success!


In [5]:
# загрузка отчета в Google Spreadsheets

gs = fr.GoogleSpreadsheet(table_name='New Report')
gs.export(df_report)

Success!


# Комментарии

In [4]:
# посмотрим на таблицу ads
sql = '''SELECT * FROM ads'''
p = fr.Postgres()
ads = p.get_df(sql)
ads.head()

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
0,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8404700756,None,2,48.840
1,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476161,None,3,100.776
2,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476162,None,1,10.140
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,None,0,0.000
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,None,0,0.000


In [6]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602 entries, 0 to 8601
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       8602 non-null   object 
 1   d_ad_account_id  8602 non-null   object 
 2   d_utm_source     8602 non-null   object 
 3   d_utm_medium     8602 non-null   object 
 4   d_utm_campaign   8602 non-null   object 
 5   d_utm_content    8602 non-null   object 
 6   d_utm_term       0 non-null      object 
 7   m_clicks         8602 non-null   int64  
 8   m_cost           8602 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 605.0+ KB


In [7]:
# смотрим наличие дубликатов
ads[['created_at', 'd_ad_account_id', 'd_utm_source', 'd_utm_medium', 'd_utm_campaign', 'd_utm_content', 'd_utm_term']]\
    .duplicated().sum()

0

In [8]:
# проверим на логическое несоответствие показателей m_clicks и m_cost
ads.query('(m_clicks==0&m_cost!=0)|(m_clicks!=0&m_cost==0)')

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost


In [9]:
ads.query('m_clicks==0')

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,None,0,0.0
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,None,0,0.0
7,2022-08-08,xo-for-client-ya,yandex,cpc,48306450,8404701663,None,0,0.0
10,2022-08-08,xo-for-client-ya,yandex,cpc,48306473,8404704716,None,0,0.0
11,2022-08-08,xo-for-client-ya,yandex,cpc,48306473,8813432765,None,0,0.0
...,...,...,...,...,...,...,...,...,...
8597,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477628,None,0,0.0
8598,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8813477629,None,0,0.0
8599,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708323,None,0,0.0
8600,2022-07-10,xo-for-client-ya,yandex,cpc,48306518,8404708324,None,0,0.0


Такие строки, где количество кликов равно нулю, нас не интересуют и в SQL запросе мы их отфильтруем, так как не являются объектом анализа. Вероятно в таблицу ads они попали по ошибке.

In [10]:
# количество уникальных значений в столбцах
ads.nunique()

created_at          254
d_ad_account_id       1
d_utm_source          1
d_utm_medium          1
d_utm_campaign       10
d_utm_content       121
d_utm_term            0
m_clicks             35
m_cost             3682
dtype: int64

Видим, что источник трафика (utm_source) единственный, так же как и тип трафика. Рекламных кампаний всего 10, им присвоены номера.

In [11]:
ads.d_utm_campaign.unique()

array(['48306435', '48306450', '48306473', '48306487', '48306494',
       '48306518', '48306461', '48306469', '60279528', '72000794'],
      dtype=object)

In [12]:
# посмотрим на таблицу leads
sql = '''SELECT * FROM leads'''
p = fr.Postgres()
leads = p.get_df(sql)
leads.head()

,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,client_id
0,2022-10-16,d8c4a9e8-4d4a-11ed-907e-848f69e142cf,None,None,None,None,None,d8c4a9e9-4d4a-11ed-907e-848f69e142cf
1,2022-10-16,c48f9767-4d4a-11ed-907e-848f69e142cf,None,None,None,None,None,c48f9768-4d4a-11ed-907e-848f69e142cf
2,2022-10-16,d74ac455-4d49-11ed-907e-848f69e142cf,vkontakte,social,kartasetevaya,None,None,f64c97d5-31ff-11ea-abeb-c412f533dba1
3,2022-10-16,0f69e77d-4d46-11ed-907e-848f69e142cf,None,None,None,None,None,None
4,2022-10-16,620ee1eb-4d45-11ed-907e-848f69e142cf,ycard,social,all,None,None,None


In [13]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23540 entries, 0 to 23539
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   lead_created_at      23540 non-null  object
 1   lead_id              23540 non-null  object
 2   d_lead_utm_source    12331 non-null  object
 3   d_lead_utm_medium    10100 non-null  object
 4   d_lead_utm_campaign  9569 non-null   object
 5   d_lead_utm_content   3338 non-null   object
 6   d_lead_utm_term      328 non-null    object
 7   client_id            18601 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


Проверим таблицу leads на дубликаты в данных. Такое могло случиться например если client_id отсутствует, среди таких строк мы не будем искать дубликаты.

In [14]:
leads[['lead_created_at', 
       'd_lead_utm_source', 
       'd_lead_utm_medium', 
       'd_lead_utm_campaign', 
       'd_lead_utm_content', 
       'd_lead_utm_term', 
       'client_id']]\
    .fillna(0).query('client_id!=0').duplicated().sum()

150

Таких строк 150. От этих дубликатов необходимо избавиться при составлении SQL запроса. 
При этом, если client_id пустое, то эти строки мы не трогаем (т.к. это могли быть разные клиенты). Если client_id не пустое, то группируем таблицу по всем столбцам, кроме lead_id, а из lead_id возьмем одно значение, например MIN.

In [15]:
# количество уникальных значений в столбцах
leads.nunique()

lead_created_at          289
lead_id                23540
d_lead_utm_source         49
d_lead_utm_medium         14
d_lead_utm_campaign      175
d_lead_utm_content       135
d_lead_utm_term           16
client_id              16164
dtype: int64

В таблице leads по сравнению с таблицей ads уже 175 рекламных кампаний, 49 источников трафика и 14 типов трафика.

In [16]:
leads.d_lead_utm_source.unique()

array([None, 'vkontakte', 'ycard', 'vk', 'vkmegacity', 'sms', 'instagram',
       'zvonobot', 'yandex', 'whatsapp', 'kviz', 'megacity', 'VOICEROBOT',
       'telegram', 'viber', 'eLama-google', 'site', 'df310722',
       'ycard#!/tproduct/323718988-1498486301712', 'google',
       'clients_day', 'inst', 'intagram', 'lift', 'banner', 'razdatka',
       'outdoor', 'promo1', 'promo5', 'promo6', 'promo3', 'promo2',
       'promo8', 'promo7', 'ff110922', 'promo11', 'pt1109', 'vkontakteso',
       'zapisnatrenirovk', 'ycard#!/tproduct/323718988-1646116631071',
       'dz1809', 'ycard#!/tproduct/323718988-1498486363994',
       'telegramsamara', 'vkmegasity', 'vkontakteknopka',
       'vkontaktereklama', 'navigaciya', 'megasity', '%utm_source%',
       'vkontaktevertical'], dtype=object)

Видим, что в d_lead_utm_source нет единого стандарта наименования источника, и если бы единственный источник в ads был бы не яндекс, а вконтакте, то данные пришлось бы приводить к единому стандарту.

In [17]:
leads.d_lead_utm_medium.unique()

array([None, 'social', 'cpc', 'email', 'organic', 'banner', 'razdatka',
       'zvonobot', 'sociak', 'cpa', 'vk', 'socia', 'banne', 'mobile',
       '%utm_medium%'], dtype=object)

Также видим погрешности в наименованиях (banner-banne, social-sociak-socia), но единственным типом трафика в ads является cpc, так что снова можем не заниматься чисткой данных.

In [18]:
leads.d_lead_utm_campaign.unique()

array([None, 'kartasetevaya', 'all', 'student', '15', '2', '900', '100',
       'fizmega', '1', '50', '78599498', '48306487', '48306435',
       '48306494', '1310', '3', '48306473', '48306450', '1rubl', 'akcii',
       '48306518', '600', 'bchk', '70', 'massage30', 'skidka_na_pt',
       'massage', '590', 'boxdety', '9000', 'outdoor', 'clients_day',
       '23849174920830725', 'Поиск II Брендовые запросы',
       '23849173065740725', '14', '30', '23849240299850725', '60279528',
       '23849255068610725', 'fitboxing', 'fizmefa', '700', 'welcome',
       'leto3', 'leto', 'lift', 'fitboxingvk', 'setevaya8', '8655725448',
       'fitnessleto', 'fitness25', 'detfitness', '8', 'zav', 'sixplussix',
       'deti290', 'denclienta', '6 6', 'inbody', 'detsifitnes', '1-3',
       'zapisnatrenirovk', 'trenirovki', '4cluba', '90', '-90', 'freemk',
       'letofitness', '3mes', '5500', '55', 'smart', '2022podarok',
       'kviz', '31-07', 'klient', '72000794', '290', '27-07-2', '27-07',
       'sale9

In [19]:
leads.d_lead_utm_content.unique()

array([None, '12793456335', '8813469521', '8813476164', '8813466151',
       '8404707872', '8404704716', '8404701661', '8813466152',
       'mainbottom', '8813432768', 'rightbottom', '8813477633',
       '8404707410', '8813469522', '8404701665', '8404701664',
       '8404701659', '8813432765', '8813466153', '8404701663',
       '8813476163', '8404707871', '8404700756', '8404708337',
       '8813469520', '8404708310', '8813477629', '8404708297',
       '8813477638', '8813469519', 'lk', '8813477631', '8404707409',
       '23849174920860725',
       'cid|8655725448|gid|87321288392|aid|406968478995|dvc|m|pid|kwd-364810444776|pos||adn|g|mt|e',
       '23849173065770725', '23849174920850725', '8813476162',
       '23849173065730725', '8404700757', '8813477626', '8813477627',
       '8813477634', '8813466154', '23849240299830725',
       'cid|8655725448|gid|87321288192|aid|406968478989|dvc|m|pid|kwd-314360287610|pos||adn|g|mt|b',
       '23849240299820725', '8813477637', '10538265370', '88134

В соответствии с этой статьей https://vc.ru/marketing/83142-gramotnye-utm-metki-dlya-skvoznoy-analitiki  
и данной в ней конструкцией ссылки  
http://example.com/?utm_source=yandex&utm_medium=cpc&utm_campaign={campaign_id}&utm_content={ad_id}&utm_term={keyword}&utmstat=us|yandex|cid|{campaign_id}|aid|{ad_id}|gid|{gbid}|pid|{phrase_id}|keyword|{keyword}|position_type|{position_type}|source|{source}|region|{region_id}|device|{device_type} я попробовала предположить, что повторяющееся "...cid|8655725448|..." - это d_lead_utm_campaign или в соответствии с аббревиатурой d_lead_utm_content на крайний случай. Таблицу leads необходимо будет соединять с таблицей ads, поэтому нас волнует, есть ли такой номер в столбцах d_utm_campaign или d_utm_content.

In [20]:
8655725448 in ads.d_utm_campaign.unique() or 8655725448 in ads.d_utm_content.unique()

False

Данный номер не встречается в столбцах campaign и content таблицы ads, значит, не будем пытаться вычленить оттуда номера.

In [21]:
# посмотрим на таблицу purchases
sql = '''SELECT * FROM purchases'''
p = fr.Postgres()
purchases = p.get_df(sql)
purchases.head()

,purchases_created_at,purchase_id,client_id,m_purchase_amount
0,2022-02-21,d3198d39-6d16-40c9-bff7-aa28bd6e2991,7011bdcd-6fd8-11e7-80fc-c412f533dba1,9950
1,2022-02-21,48a0ad24-77aa-4064-a971-dd0d6f1f6c50,a2771bb0-6fd6-11e7-80fc-c412f533dba1,8700
2,2022-02-21,267ff20f-a56c-480a-b2f7-1f7b1a746f9f,2687f503-6fd7-11e7-80fc-c412f533dba1,0
3,2022-02-21,19fba5f0-785d-45bd-903a-34ba529c6404,01122a97-45bc-11eb-ac23-c412f533dba1,8700
4,2022-02-21,00ac921d-32e3-4200-9435-a46d5fef4a29,a55b8c4e-d5a3-11e9-abbe-c412f533dba1,500


In [22]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66180 entries, 0 to 66179
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   purchases_created_at  66180 non-null  object
 1   purchase_id           66180 non-null  object
 2   client_id             66179 non-null  object
 3   m_purchase_amount     66180 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.0+ MB


Есть один клиент с пустым id, но при объединении таблиц он отсеется.

In [23]:
# покупки с нулевой суммой
purchases.query('m_purchase_amount==0')

,purchases_created_at,purchase_id,client_id,m_purchase_amount
2,2022-02-21,267ff20f-a56c-480a-b2f7-1f7b1a746f9f,2687f503-6fd7-11e7-80fc-c412f533dba1,0
11,2022-02-22,d218cfdb-aaf0-418f-bd46-0c734388e7ab,dbce39b8-64d3-11e9-ab9f-c412f533dba1,0
22,2022-02-28,d7934a35-36b0-4eba-a066-cc1b7f83d431,84a77a5d-96f6-11ec-905d-848f69e142cf,0
30,2022-07-06,584c296b-c21c-4284-a8aa-f60c1ecffb84,98bb3432-1db6-11ea-abea-c412f533dba1,0
35,2022-03-15,937260f1-d800-4000-b451-05e028cb8b50,48096567-9493-11e7-8114-c412f533dba1,0
...,...,...,...,...
66160,2022-02-07,a396f3c1-27b5-49a4-b749-fa0816b9456f,9ffd4518-6fd8-11e7-80fc-c412f533dba1,0
66164,2022-02-15,756fca55-792a-4d76-ba52-9c0c559a549f,c6d96d70-57fd-11ec-9054-848f69e142cf,0
66165,2022-02-15,a9bec4a7-fd58-44d2-8fe5-df0229e1e8a6,20c3a2f4-0330-11eb-ac18-c412f533dba1,0
66168,2022-02-11,bd4eeb9c-a224-4934-8c04-dee472c4e03f,89e363a3-8b04-11ec-905a-848f69e142cf,0


В SQL запросе необходимо отфильтровать строки, где m_purchase_amount = 0. Возможно это были отмененные покупки или получение чего-то бесплатного (напр. по акции) через сайт, что не имеет отношения к прибыли и оценке эффективности маркетинговой кампании в рамках данного сквозного анализа.
Также при объединении таблиц необходимо верно атрибутировать продажи к лидам. Это можно осуществить через оконные функции SQL. 